## 0. Xử lý data

In [1]:
! pip install pandas numpy prophet tensorflow scikit-learn matplotlib

In [2]:
import pandas as pd

# Đường dẫn file
input_path = 'E:/vietlott_new\data_mega\data.csv'

# Đọc dữ liệu với dấu phân tách tab
try:
    df = pd.read_csv(
        input_path,
        encoding='utf-8',
        sep='\t',  # Sử dụng dấu tab làm phân tách
        on_bad_lines='skip',  # Bỏ qua dòng lỗi
        quoting=3,  # Không xử lý dấu ngoặc kép
        dtype={'Giải Jackpot': str}  # Cột Giải Jackpot là chuỗi
    )
except Exception as e:
    print(f"Lỗi khi đọc file: {e}")
    exit()

# Loại bỏ khoảng trắng thừa trong tên cột
df.columns = df.columns.str.strip()

# 1. Xác định số cột và số hàng
num_rows, num_cols = df.shape
print(f"Số hàng: {num_rows}")
print(f"Số cột: {num_cols}")
print("\nTên các cột:", list(df.columns))

# 2. In 20 hàng đầu tiên
print("\n20 hàng đầu tiên:")
print(df.head(20).to_string(index=False))

# 3. In 20 hàng cuối cùng
print("\n20 hàng cuối cùng:")
print(df.tail(20).to_string(index=False))

# 4. Làm sạch dữ liệu (kiểm tra giá trị thiếu)
print("\nThông tin dữ liệu:")
print(df.info())
print("\nSố giá trị thiếu trong mỗi cột:")
print(df.isnull().sum())

# 5. Chuẩn bị dữ liệu cho phân tích tiếp theo
# Tách cột Kết Quả thành 6 số riêng biệt
def parse_results(result):
    try:
        return [int(x) for x in result.split()]
    except:
        return [None] * 6

df['Numbers'] = df['Kết Quả'].apply(parse_results)
df[['Num1', 'Num2', 'Num3', 'Num4', 'Num5', 'Num6']] = pd.DataFrame(df['Numbers'].tolist(), index=df.index)

# Kiểm tra dữ liệu sau khi tách
print("\nDữ liệu sau khi tách cột Kết Quả (5 hàng đầu tiên):")
print(df[['Ngày Mở Thưởng', 'Num1', 'Num2', 'Num3', 'Num4', 'Num5', 'Num6']].head().to_string(index=False))

Số hàng: 1366
Số cột: 3

Tên các cột: ['Ngày Mở Thưởng', 'Kết Quả', 'Giải Jackpot']

20 hàng đầu tiên:
Ngày Mở Thưởng           Kết Quả   Giải Jackpot
T6, 13/06/2025 01 13 19 28 30 36 68.541.913.000
T4, 11/06/2025 10 13 22 27 41 45 62.026.776.000
CN, 08/06/2025 06 17 18 19 31 37 55.650.339.500
T6, 06/06/2025 24 26 35 39 41 45 49.708.989.500
T4, 04/06/2025 11 23 26 27 29 32 44.017.957.000
CN, 01/06/2025 09 24 29 37 42 44 38.847.124.000
T6, 30/05/2025 04 07 12 35 37 41 34.305.979.500
T4, 28/05/2025 17 22 23 28 31 41 29.736.250.000
CN, 25/05/2025 05 14 23 24 28 44 25.290.182.500
T6, 23/05/2025 02 04 15 16 29 43 23.051.056.000
T4, 21/05/2025 02 13 14 21 24 30 20.908.114.000
CN, 18/05/2025 06 07 17 27 30 42 18.835.213.500
T6, 16/05/2025 05 09 14 22 32 44 16.981.638.500
T4, 14/05/2025 19 24 27 37 44 45 15.364.240.000
CN, 11/05/2025 13 15 17 22 28 43 13.687.214.500
T6, 09/05/2025 01 07 09 34 40 42 78.231.289.000
T4, 07/05/2025 08 27 28 31 34 40 71.466.517.000
CN, 04/05/2025 06 07 08 09 12 29 

In [3]:
import pandas as pd

# Định nghĩa đường dẫn đến tệp CSV của bạn
input_path = 'E:/vietlott_new\data_mega\data.csv'

try:
    df = pd.read_csv(input_path, encoding='utf-8', sep='\t', on_bad_lines='skip', quoting=3)
    print("Tệp được tải thành công. Các cột:", list(df.columns))
except FileNotFoundError:
    print(f"Lỗi: Không tìm thấy tệp tại {input_path}")
except Exception as e:
    print(f"Lỗi khi tải tệp: {e}")

Tệp được tải thành công. Các cột: ['Ngày Mở Thưởng', 'Kết Quả', 'Giải Jackpot']


In [4]:
import pandas as pd
import numpy as np
import logging
from datetime import datetime

# Thiết lập logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Đường dẫn file
input_path = 'E:/vietlott_new\data_mega\data.csv'
output_path = 'E:/vietlott_new\data_mega\data_clean.csv'

# Đọc dữ liệu
try:
    df = pd.read_csv(
        input_path,
        encoding='utf-8',
        sep='\t',
        on_bad_lines='skip',
        quoting=3,
        dtype={'Giải Jackpot': str}
    )
    logging.info("Tệp được tải thành công. Danh sách cột: %s", list(df.columns))
except FileNotFoundError:
    logging.error("Không tìm thấy tệp tại %s", input_path)
    exit()
except Exception as e:
    logging.error("Lỗi khi đọc file: %s", e)
    exit()

# Làm sạch tên cột
df.columns = [col.strip() for col in df.columns]

# Hàm tách cột Kết Quả thành 6 số và kiểm tra tính hợp lệ
def parse_results(result):
    try:
        numbers = [int(x) for x in str(result).split()]
        # Kiểm tra số lượng và giá trị hợp lệ (1-45)
        if len(numbers) == 6 and all(1 <= x <= 45 for x in numbers):
            return numbers
        return [np.nan] * 6
    except:
        return [np.nan] * 6

# Tách cột Kết Quả
if 'Kết Quả' in df.columns:
    df['Numbers'] = df['Kết Quả'].apply(parse_results)
    df[['Num1', 'Num2', 'Num3', 'Num4', 'Num5', 'Num6']] = pd.DataFrame(df['Numbers'].tolist(), index=df.index)
    df = df.drop(columns=['Numbers', 'Kết Quả'], errors='ignore')
else:
    logging.error("Cột 'Kết Quả' không tồn tại")
    exit()

# Xử lý cột Ngày Mở Thưởng
def parse_date(date_str):
    try:
        # Chuyển định dạng T6, 13/06/2025 sang datetime
        date_str = date_str.split(', ')[1]
        return pd.to_datetime(date_str, format='%d/%m/%Y')
    except:
        return pd.NaT

df['Ngày Mở Thưởng'] = df['Ngày Mở Thưởng'].apply(parse_date)

# Xử lý cột Giải Jackpot
def clean_jackpot(jackpot):
    try:
        # Loại bỏ dấu chấm và chuyển sang số nguyên
        return int(jackpot.replace('.', ''))
    except:
        return np.nan

df['Giải Jackpot'] = df['Giải Jackpot'].apply(clean_jackpot)

# Kiểm tra và loại bỏ hàng có giá trị thiếu
initial_rows = df.shape[0]
df = df.dropna()
logging.info("Loại bỏ %d hàng có giá trị thiếu", initial_rows - df.shape[0])

# Kiểm tra trùng lặp
duplicates = df.duplicated().sum()
if duplicates > 0:
    df = df.drop_duplicates()
    logging.info("Loại bỏ %d hàng trùng lặp", duplicates)

# Sắp xếp dữ liệu theo ngày
df = df.sort_values('Ngày Mở Thưởng', ascending=False)

# Sắp xếp lại thứ tự cột
df = df[['Ngày Mở Thưởng', 'Num1', 'Num2', 'Num3', 'Num4', 'Num5', 'Num6', 'Giải Jackpot']]

# In thông tin dữ liệu
logging.info("Số hàng: %d", df.shape[0])
logging.info("Số cột: %d", df.shape[1])
logging.info("Tên các cột: %s", list(df.columns))
logging.info("\n5 hàng đầu tiên:\n%s", df.head().to_string(index=False))

# Kiểm tra giá trị thiếu
logging.info("\nSố giá trị thiếu trong mỗi cột:\n%s", df.isnull().sum())

# Lưu dữ liệu sạch
try:
    df.to_csv(output_path, index=False, encoding='utf-8', sep='\t')
    logging.info("Dữ liệu đã được lưu vào %s", output_path)
except Exception as e:
    logging.error("Lỗi khi lưu file: %s", e)

# Chuẩn bị dữ liệu cho mô hình dự đoán
# Thêm các cột đặc trưng (feature engineering)
df['Sum'] = df[['Num1', 'Num2', 'Num3', 'Num4', 'Num5', 'Num6']].sum(axis=1)
df['Mean'] = df[['Num1', 'Num2', 'Num3', 'Num4', 'Num5', 'Num6']].mean(axis=1)
df['Std'] = df[['Num1', 'Num2', 'Num3', 'Num4', 'Num5', 'Num6']].std(axis=1)

# Lưu dữ liệu với các đặc trưng bổ sung
output_features_path = 'E:/vietlott_new\data_mega\data_clean_with_features.csv'
try:
    df.to_csv(output_features_path, index=False, encoding='utf-8', sep='\t')
    logging.info("Dữ liệu với các đặc trưng bổ sung đã được lưu vào %s", output_features_path)
except Exception as e:
    logging.error("Lỗi khi lưu file đặc trưng: %s", e)

2025-06-21 14:05:21,983 - INFO - Tệp được tải thành công. Danh sách cột: ['Ngày Mở Thưởng', 'Kết Quả', 'Giải Jackpot']
2025-06-21 14:05:22,557 - INFO - Loại bỏ 1 hàng có giá trị thiếu
2025-06-21 14:05:22,639 - INFO - Số hàng: 1365
2025-06-21 14:05:22,641 - INFO - Số cột: 8
2025-06-21 14:05:22,641 - INFO - Tên các cột: ['Ngày Mở Thưởng', 'Num1', 'Num2', 'Num3', 'Num4', 'Num5', 'Num6', 'Giải Jackpot']
2025-06-21 14:05:22,645 - INFO - 
5 hàng đầu tiên:
Ngày Mở Thưởng  Num1  Num2  Num3  Num4  Num5  Num6  Giải Jackpot
    2025-06-13     1    13    19    28    30    36  6.854191e+10
    2025-06-11    10    13    22    27    41    45  6.202678e+10
    2025-06-08     6    17    18    19    31    37  5.565034e+10
    2025-06-06    24    26    35    39    41    45  4.970899e+10
    2025-06-04    11    23    26    27    29    32  4.401796e+10
2025-06-21 14:05:22,647 - INFO - 
Số giá trị thiếu trong mỗi cột:
Ngày Mở Thưởng    0
Num1              0
Num2              0
Num3              0
Num4      

## 1. Phương pháp thống kê
### Dựa trên tần suất + Dựa trên cặp số + Dựa trên chu kỳ:

In [5]:
import pandas as pd
import numpy as np
from collections import Counter
from itertools import combinations
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import f1_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping
import logging
from datetime import datetime, timedelta

# Thiết lập logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Đường dẫn file dữ liệu
input_path = 'E:/vietlott_new/data_mega/data_clean.csv'.replace('\\', '/')

# Hàm đọc dữ liệu
def load_data(path):
    try:
        df = pd.read_csv(path, encoding='utf-8', sep='\t')
        if df.empty:
            logging.error("Dữ liệu rỗng hoặc không đọc được.")
            return None
        return df
    except FileNotFoundError:
        logging.error("Không tìm thấy file: %s", path)
        return None
    except Exception as e:
        logging.error("Lỗi khi đọc file: %s", e)
        return None

# Đọc dữ liệu
df = load_data(input_path)
if df is None:
    exit()

# Kiểm tra cột cần thiết
required_columns = ['Ngày Mở Thưởng', 'Num1', 'Num2', 'Num3', 'Num4', 'Num5', 'Num6']
if not all(col in df.columns for col in required_columns):
    logging.error("File CSV thiếu các cột cần thiết: %s", required_columns)
    exit()

# Chuyển đổi cột ngày sang datetime
df['Ngày Mở Thưởng'] = pd.to_datetime(df['Ngày Mở Thưởng'], format='%Y-%m-%d')

# Xác định ngày kỳ quay tiếp theo
latest_date = df['Ngày Mở Thưởng'].max()  # 13/06/2025
def get_next_draw_date(current_date):
    return current_date + timedelta(days=2)  # Thứ Sáu -> Chủ Nhật (15/06/2025)

next_draw_date = get_next_draw_date(latest_date)
logging.info("Ngày mới nhất trong dữ liệu: %s", latest_date.strftime('%Y-%m-%d'))
logging.info("Kỳ quay tiếp theo: %s", next_draw_date.strftime('%Y-%m-%d'))

# Lấy dữ liệu số
numbers_df = df[['Num1', 'Num2', 'Num3', 'Num4', 'Num5', 'Num6']]

# --- Phân tích đặc trưng ---
# 1. Tần suất
def frequency_analysis(df, recent=False):
    df = df.tail(90) if recent and len(df) > 90 else df
    all_numbers = df.values.flatten()
    freq = Counter(n for n in all_numbers if 1 <= n <= 45)
    sorted_freq = sorted(freq.items(), key=lambda x: x[1], reverse=True)
    return sorted_freq[:6], sorted_freq[:10], freq

# 2. Cặp số
def pair_analysis(df):
    pairs = [tuple(sorted(row.values)) for _, row in df.iterrows()]
    pair_freq = Counter(p for p in (combinations(c, 2) for c in pairs) for p in p if all(1 <= x <= 45 for x in p))
    sorted_pairs = sorted(pair_freq.items(), key=lambda x: x[1], reverse=True)
    if not sorted_pairs:
        return [], []
    pair = sorted_pairs[0][0]
    related = []
    for _, row in df.iterrows():
        row_set = set(row.values)
        if set(pair).issubset(row_set):
            related.extend(n for n in row_set if n not in pair and 1 <= n <= 45)
    related_freq = Counter(related)
    top_related = [n for n, _ in sorted(related_freq.items(), key=lambda x: x[1], reverse=True)[:4]]
    predicted = sorted(list(pair) + top_related)
    return predicted[:6], sorted_pairs[:5]

# 3. Chu kỳ
def cycle_analysis(df):
    cycles = {i: [] for i in range(1, 46)}
    for idx, row in df.iterrows():
        for num in row.values:
            if 1 <= num <= 45:
                cycles[num].append(idx)
    predicted = []
    current_row = len(df) - 1
    for num in range(1, 46):
        cycle = cycles[num]
        if not cycle:
            continue
        gaps = np.diff(cycle)
        mean_gap = np.mean(gaps) if len(gaps) > 0 else float('inf')
        if mean_gap > 0 and cycle:
            cycles_since_last = current_row - cycle[-1]
            likelihood = min(cycles_since_last / mean_gap, 1.0)
            predicted.append((num, likelihood))
    predicted.sort(key=lambda x: x[1], reverse=True)
    return [n for n, _ in predicted[:6]], predicted[:10]

# 4. Nhóm số
def cluster_analysis(df):
    clusters = {'Thấp': range(1, 16), 'Trung': range(16, 31), 'Cao': range(31, 46)}
    counts = []
    for _, row in df.iterrows():
        c = {'Thấp': 0, 'Trung': 0, 'Cao': 0}
        for n in row.values:
            if n in clusters['Thấp']:
                c['Thấp'] += 1
            elif n in clusters['Trung']:
                c['Trung'] += 1
            elif n in clusters['Cao']:
                c['Cao'] += 1
        counts.append(c)
    return pd.DataFrame(counts).mean()

# 5. Chẵn/lẻ và tổng số
def even_odd_analysis(df):
    ratios = []
    sums = []
    for _, row in df.iterrows():
        valid_nums = [n for n in row.values if 1 <= n <= 45]
        even_count = sum(1 for n in valid_nums if n % 2 == 0)
        ratios.append(f"{even_count}/{6-even_count}")
        sums.append(sum(valid_nums))
    return Counter(ratios).most_common(5), np.mean(sums) if sums else 0

# 6. Kết hợp dự đoán
def combine_predictions(freq, pair, cycle, recent):
    all_nums = [n for n, _ in freq] + pair + cycle + [n for n, _ in recent]
    combined = Counter(n for n in all_nums if 1 <= n <= 45)
    sorted_combined = sorted(combined.items(), key=lambda x: (x[1], x[0]), reverse=True)
    return sorted([n for n, _ in sorted_combined[:6]])

# --- Mô hình LSTM ---
def prepare_lstm_data(df, lags=7):
    data = df[['Num1', 'Num2', 'Num3', 'Num4', 'Num5', 'Num6']].values
    scaler = MinMaxScaler()
    data_scaled = scaler.fit_transform(data.reshape(-1, 6)).reshape(-1, 6)
    
    X, y = [], []
    for i in range(lags, len(data_scaled)):
        X.append(data_scaled[i-lags:i])
        y.append(data_scaled[i])
    X, y = np.array(X), np.array(y)
    
    train_size = int(0.8 * len(X))
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]
    
    return X_train, y_train, X_test, y_test, scaler

def train_lstm(X_train, y_train, X_test, y_test):
    model = Sequential([
        Input(shape=(X_train.shape[1], X_train.shape[2])),
        LSTM(128, return_sequences=True),
        Dropout(0.2),
        LSTM(64),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dense(6, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='mse')
    
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    
    history = model.fit(
        X_train, y_train,
        epochs=200,
        batch_size=32,
        validation_data=(X_test, y_test),
        callbacks=[early_stopping],
        verbose=0
    )
    
    return model, history

def evaluate_f1_score(y_true, y_pred):
    y_true = np.round(y_true).astype(int).flatten()
    y_pred = np.round(y_pred).astype(int).flatten()
    return f1_score(y_true, y_pred, average='micro', zero_division=0)

def lstm_prediction(df, lags=7):
    X_train, y_train, X_test, y_test, scaler = prepare_lstm_data(df, lags)
    model, history = train_lstm(X_train, y_train, X_test, y_test)
    
    y_pred_scaled = model.predict(X_test, verbose=0)
    y_pred = scaler.inverse_transform(y_pred_scaled)
    f1 = evaluate_f1_score(scaler.inverse_transform(y_test), y_pred)
    final_loss = history.history['loss'][-1]
    final_val_loss = history.history['val_loss'][-1]
    logging.info("LSTM - Số epoch thực tế: %d", len(history.history['loss']))
    logging.info("LSTM - Loss cuối: %.4f, Validation Loss cuối: %.4f", final_loss, final_val_loss)
    logging.info("LSTM - F1-score trên tập test: %.4f", f1)
    
    last_sequence = df[['Num1', 'Num2', 'Num3', 'Num4', 'Num5', 'Num6']].values[-lags:]
    last_sequence_scaled = scaler.transform(last_sequence)
    last_sequence_scaled = last_sequence_scaled.reshape(1, lags, 6)
    
    pred_scaled = model.predict(last_sequence_scaled, verbose=0)
    pred = scaler.inverse_transform(pred_scaled)
    pred = np.round(pred).astype(int)
    
    pred = np.clip(pred, 1, 45)
    pred = np.unique(pred)
    while len(pred) < 6:
        pred = np.append(pred, np.random.randint(1, 46))
    return sorted(pred[:6]), f1, final_loss, final_val_loss

# Thực hiện phân tích
freq_nums, freq_details, _ = frequency_analysis(numbers_df)
recent_nums, recent_details, _ = frequency_analysis(numbers_df, recent=True)
pair_nums, pair_details = pair_analysis(numbers_df)
cycle_nums, cycle_details = cycle_analysis(numbers_df)
cluster_avg = cluster_analysis(numbers_df)
even_odd_ratios, avg_sum = even_odd_analysis(numbers_df)
final_nums = combine_predictions(freq_nums, pair_nums, cycle_nums, recent_nums)
lstm_pred, f1_score_lstm, final_loss, final_val_loss = lstm_prediction(numbers_df)

# Đánh giá mô hình
logging.info("\nĐánh giá mô hình:")
logging.info("Tần suất, cặp số, chu kỳ, kết hợp: Dựa trên thống kê, không có F1-score.")
logging.info("LSTM: F1-score = %.4f, Loss = %.4f, Validation Loss = %.4f", f1_score_lstm, final_loss, final_val_loss)
logging.info("Nhận xét: F1-score thấp là bình thường do tính ngẫu nhiên. F1 > 0.1 là khả quan.")

# In kết quả
logging.info(f"\nDỰ ĐOÁN KỲ QUAY VIETLOTT MEGA 6/45 ({next_draw_date.strftime('%Y-%m-%d %H:%M:%S')})\n")

logging.info("1. PHÂN TÍCH NHÓM SỐ")
logging.info("Tỷ lệ trung bình mỗi nhóm:\n%s", cluster_avg.to_string())
logging.info("")

logging.info("2. PHÂN TÍCH CHẴN/LẺ")
logging.info("Tỷ lệ chẵn/lẻ phổ biến:")
for r, c in even_odd_ratios:
    logging.info("%s: %d lần", r, c)
logging.info("Tổng trung bình: %.2f", avg_sum)
logging.info("")

logging.info("3. DỰ ĐOÁN DỰA TRÊN TẦN SUẤT")
logging.info("Bộ số dự đoán: %s", [n for n, _ in freq_nums])
logging.info("Chi tiết (Top 10):")
for n, c in freq_details:
    logging.info("Số %d: %d lần", n, c)
logging.info("")

logging.info("4. DỰ ĐOÁN DỰA TRÊN TẦN SUẤT GẦN ĐÂY (3 tháng)")
logging.info("Bộ số dự đoán: %s", [n for n, _ in recent_nums])
logging.info("Chi tiết (Top 10):")
for n, c in recent_details:
    logging.info("Số %d: %d lần", n, c)
logging.info("")

logging.info("5. DỰ ĐOÁN DỰA TRÊN CẶP SỐ")
logging.info("Bộ số dự đoán: %s", pair_nums)
logging.info("Chi tiết cặp số (Top 5):")
for p, c in pair_details:
    logging.info("Cặp %s: %d lần", p, c)
logging.info("")

logging.info("6. DỰ ĐOÁN DỰA TRÊN CHU KỲ")
logging.info("Bộ số dự đoán: %s", cycle_nums)
logging.info("Chi tiết (Top 10):")
for n, l in cycle_details:
    logging.info("Số %d: Xác suất chu kỳ = %.2f", n, l)
logging.info("")

logging.info("7. BỘ SỐ DỰ ĐOÁN KẾT HỢP")
logging.info("Bộ số cuối cùng: %s", final_nums)
logging.info("")

logging.info("8. DỰ ĐOÁN DỰA TRÊN LSTM")
logging.info("Bộ số dự đoán: %s", lstm_pred)
logging.info("F1-score: %.4f, Loss: %.4f, Val Loss: %.4f", f1_score_lstm, final_loss, final_val_loss)
logging.info("\nLưu ý: Dự đoán chỉ mang tính tham khảo. Chúc may mắn!\n")

2025-06-21 14:05:28,499 - INFO - Ngày mới nhất trong dữ liệu: 2025-06-13
2025-06-21 14:05:28,500 - INFO - Kỳ quay tiếp theo: 2025-06-15
2025-06-21 14:05:41,051 - INFO - LSTM - Số epoch thực tế: 18
2025-06-21 14:05:41,053 - INFO - LSTM - Loss cuối: 0.0340, Validation Loss cuối: 0.0360
2025-06-21 14:05:41,055 - INFO - LSTM - F1-score trên tập test: 0.0521
2025-06-21 14:05:41,152 - INFO - 
Đánh giá mô hình:
2025-06-21 14:05:41,154 - INFO - Tần suất, cặp số, chu kỳ, kết hợp: Dựa trên thống kê, không có F1-score.
2025-06-21 14:05:41,155 - INFO - LSTM: F1-score = 0.0521, Loss = 0.0340, Validation Loss = 0.0360
2025-06-21 14:05:41,155 - INFO - Nhận xét: F1-score thấp là bình thường do tính ngẫu nhiên. F1 > 0.1 là khả quan.
2025-06-21 14:05:41,158 - INFO - 
DỰ ĐOÁN KỲ QUAY VIETLOTT MEGA 6/45 (2025-06-15 00:00:00)

2025-06-21 14:05:41,159 - INFO - 1. PHÂN TÍCH NHÓM SỐ
2025-06-21 14:05:41,161 - INFO - Tỷ lệ trung bình mỗi nhóm:
Thấp     2.007326
Trung    2.041026
Cao      1.951648
2025-06-21 14:

####